# ✨ UnQCreator Engine Launcher

Welcome to the UnQCreator Engine Launcher! This notebook sets up your cloud-based engine backend for creating amazing videos.

## 🚀 Quick Start:
1. Enter your Vercel URL and Cloudflare Token in the form below
2. Click the "Launch Engine" button in the next cell
3. Wait for the setup to complete (5-10 minutes)
4. You'll receive your public API endpoint when ready

## 🔗 Connect Your Frontend:
Once the engine is running, update your frontend settings to point to your new API endpoint.

In [ ]:
#@title 💾 1. Configuration
#@markdown ---
#@markdown Enter your Vercel frontend URL (for security/CORS) and your Cloudflare Tunnel Token.
#@markdown You can get your Cloudflare token from the Zero Trust dashboard.
VERCEL_URL = "https://unqcreator.vercel.app" #@param {type:"string"}
CLOUDFLARE_TOKEN = "" #@param {type:"string"}

if not CLOUDFLARE_TOKEN:
  print("❌ Error: Please enter your Cloudflare Token.")
else:
  print("✅ Configuration loaded successfully.")

In [ ]:
#@title 🚀 2. Launch Engine
#@markdown Click the play button to start the UnQCreator Engine setup process.

import os
import time
import asyncio
from subprocess import Popen
from IPython.display import HTML, display

# Display a stylish progress indicator
def show_progress(message):
    print(f"\033[1;34m{message}\033[0m")
    
# Step 1: Clone the repository
show_progress("📥 Cloning the UnQCreator repository...")
!git clone https://github.com/Sandeepgaddam5432/unqcreator.git /content/unqcreator
%cd /content/unqcreator
show_progress("✅ Repository cloned successfully!")

# Step 2: Install dependencies with correct path to requirements.txt
show_progress("📦 Installing Python dependencies... (This may take a few minutes)")
!pip install -q -r video_genarater/UnQ-YT-Auto/requirements.txt
show_progress("✅ Dependencies installed successfully!")

# Step 3: Setup Cloudflare Tunnel
show_progress("🔒 Setting up Cloudflare Tunnel...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
show_progress("✅ Cloudflare client downloaded and installed!")

# Define the async function to run the cloudflared tunnel
async def run_cloudflared():
    show_progress("🚇 Starting Cloudflare Tunnel...")
    os.environ['CLOUDFLARE_TOKEN'] = CLOUDFLARE_TOKEN
    # This will block, so we run it in a way that lets the notebook continue
    process = Popen(['cloudflared', 'tunnel', 'run', '--token', CLOUDFLARE_TOKEN])
    show_progress(f"✅ Cloudflare Tunnel established! (Process ID: {process.pid})")

# Launch the Cloudflare tunnel in the background
asyncio.create_task(run_cloudflared())
time.sleep(3)  # Give cloudflared a moment to start

# Display final setup message
show_progress("\n🎬 Starting UnQCreator Engine...")
show_progress(f"🔐 Allowing requests only from: {VERCEL_URL}")

# Show pretty note box
display(HTML("""
<div style="background-color: #f0f7ff; padding: 15px; border-left: 5px solid #0366d6; border-radius: 5px; margin: 20px 0;">
  <h3 style="margin-top: 0; color: #0366d6;">🎉 Your Engine is Starting!</h3>
  <p>The UnQCreator Engine will be available at a <code>.trycloudflare.com</code> URL shortly.</p>
  <p>Copy this URL and paste it into your frontend Settings page.</p>
</div>
"""))

# Launch the server with the correct path
print("\n📡 Engine output will appear below:\n")
print("------------------------------------------")
!cd video_genarater/UnQ-YT-Auto && python main.py --listen --enable-cors-header "{VERCEL_URL}"